In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.layered.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.lqns.*
import jline.solvers.ln.*
import jline.solvers.nc.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.SILENT)

println("This example illustrates the solution of a moderately large LQN.")

// Create a representative large layered model (ofbizExample substitute)
val model = LayeredNetwork("LargeLayeredModel")

// Create multiple processors (representative of ofbiz complexity)
val processors = mutableMapOf<Int, Processor>()
for (i in 1..5) {
    processors[i] = Processor(model, "P$i", i.toDouble(), SchedStrategy.PS)
}

In [ ]:
// Create multiple tasks (representative structure)
val tasks = mutableMapOf<Int, Task>()
val entries = mutableMapOf<Int, Entry>()

// Client layer
tasks[1] = Task(model, "ClientTask", 10, SchedStrategy.REF).on(processors[1]!!).setThinkTime(Exp.fitMean(1.0))
entries[1] = Entry(model, "ClientEntry").on(tasks[1]!!)

// Service layers
for (i in 2..4) {
    tasks[i] = Task(model, "ServiceTask$i", 1, SchedStrategy.FCFS).on(processors[i]!!).setThinkTime(Immediate())
    entries[i] = Entry(model, "ServiceEntry$i").on(tasks[i]!!)
}

// Database layer
tasks[5] = Task(model, "DatabaseTask", 1, SchedStrategy.FCFS).on(processors[5]!!).setThinkTime(Immediate())
entries[5] = Entry(model, "DatabaseEntry").on(tasks[5]!!)

In [ ]:
// Define activities and call structure (representative of ofbiz complexity)
val activities = mutableMapOf<Int, Activity>()

// Client activities
activities[1] = Activity(model, "ClientActivity", Exp.fitMean(0.1)).on(tasks[1]!!).boundTo(entries[1]!!)
activities[2] = Activity(model, "ClientCall1", Immediate()).on(tasks[1]!!).synchCall(entries[2]!!, 1.0)
activities[3] = Activity(model, "ClientCall2", Immediate()).on(tasks[1]!!).synchCall(entries[3]!!, 1.0)
activities[4] = Activity(model, "ClientCall3", Immediate()).on(tasks[1]!!).synchCall(entries[4]!!, 1.0)

// Service activities
for (i in 2..4) {
    val serviceIdx = 5 + (i-2)*2
    val dbCallIdx = 6 + (i-2)*2
    activities[serviceIdx] = Activity(model, "ServiceActivity$i", Exp.fitMean(0.5 + i * 0.2)).on(tasks[i]!!).boundTo(entries[i]!!)
    activities[dbCallIdx] = Activity(model, "DBCall$i", Immediate()).on(tasks[i]!!).synchCall(entries[5]!!, 1.0).repliesTo(entries[i]!!)
    tasks[i]!!.addPrecedence(ActivityPrecedence.Serial(activities[serviceIdx]!!, activities[dbCallIdx]!!))
}

// Database activity
activities[11] = Activity(model, "DatabaseActivity", Exp.fitMean(2.0)).on(tasks[5]!!).boundTo(entries[5]!!).repliesTo(entries[5]!!)

In [ ]:
// Add precedences for client task
tasks[1]!!.addPrecedence(ActivityPrecedence.Serial(activities[1]!!, activities[2]!!))
tasks[1]!!.addPrecedence(ActivityPrecedence.Serial(activities[2]!!, activities[3]!!))
tasks[1]!!.addPrecedence(ActivityPrecedence.Serial(activities[3]!!, activities[4]!!))

println("Large layered model created (ofbizExample substitute)")

In [ ]:
// Set options
val options = LQNS.defaultOptions()
options.keep = true // Keep intermediate XML files

// Solve with LQNS
val solvers = mutableMapOf<Int, NetworkSolver>()
val avgTables = mutableMapOf<Int, Any>()

if (LQNS.isAvailable()) {
    solvers[1] = LQNS(model)
    try {
        avgTables[1] = solvers[1]!!.avgTable
        println("AvgTable[1] (LQNS):")
        (avgTables[1] as AvgTable).print()
    } catch (e: Exception) {
        println("LQNS solver error: ${e.message}")
    }
} else {
    println("LQNS solver not available - skipping solver[1]")
}

In [ ]:
// Solve with LN without initialization
val ncOptions = NC.defaultOptions()
ncOptions.verbose = false
solvers[2] = LN(model) { x -> NC(x, ncOptions) }

val startTime = System.currentTimeMillis()
try {
    avgTables[2] = solvers[2]!!.avgTable
    val endTime = System.currentTimeMillis()
    val timeElapsed = (endTime - startTime) / 1000.0
    
    println("AvgTable[2] (LN with NC):")
    (avgTables[2] as AvgTable).print()
    println("Time elapsed: ${timeElapsed}s")
} catch (e: Exception) {
    println("LN solver error: ${e.message}")
}